# Scraping modulargrid.net

## Libraries

In [288]:
import json
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import cloudinary
import cloudinary.uploader

## Functions

In [289]:
def wait_for_scroll(old_height, timeout = 10):

    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script("return document.body.scrollHeight") > old_height
        
    )

In [302]:
def scrape_page(module_info, wait, driver):

    url = module_info['link']
    module_id = module_info['id']

    try: 
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".module-view-header h1")))
    except Exception as e:
        print(f"Timeout waiting for module name on {url}: {e}")
    
    #--------------------------------------- MODULE NAME ---------------------------------------
    try:
        module_name_element = driver.find_element(By.CSS_SELECTOR, ".module-view-header h1")
        module_name = module_name_element.text.strip()
    except Exception as e:
        print(f"Module name not found for {url}: {e}")
        module_name = ""

    # --------------------------------------- MANUFACTURER ---------------------------------------
    try: 
        manufacturer_element = driver.find_element(By.CSS_SELECTOR, ".vendor-name")
        manufacturer = manufacturer_element.text.strip()
    except Exception as e:
        print(f"Manufacturer not found for {url}: {e}")
        manufacturer = ""
    
    # --------------------------------------- PRIMARY DESCRIPTION ---------------------------------------
    try:
        primary_description_element = driver.find_element(By.CSS_SELECTOR, "p.lead.wrap")
        primary_description = primary_description_element.text.strip()
    except Exception as e:
        print(f"Primary description not found for {url}: {e}")
        primary_description = ""

    # --------------------------------------- AVAILABILITY ---------------------------------------
    try: 
        availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'subspec')]/p[contains(@class, 'text-success') and contains(., 'currently available')]")
        availability = 1 if availability_elements else 0
    except Exception as e:
        print(f"Error determining availability for {url}: {e}")
        availability = 0

    # --------------------------------------- APPROVED STAMP ---------------------------------------
    try:
        driver.find_element(By.CSS_SELECTOR, ".box-approved")
        approved_stamp = 1
    except Exception:
        approved_stamp = 0
    
    # --------------------------------------- PHYSICAL DIMENSIONS ---------------------------------------
    try:
        dims_dt = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Dimensions']")
        parent_dl = dims_dt.find_element(By.XPATH, "./..")
        dd_elements = parent_dl.find_elements(By.TAG_NAME, "dd")
        dimensions_list = [dd.text.strip() for dd in dd_elements if dd.text.strip()]
        dimensions = " | ".join(dimensions_list)
    except Exception as e:
        print(f"Dimensions not found for {url}: {e}")
        dimensions = ""

    # --------------------------------------- POWER REQUIREMENTS ---------------------------------------
    try:
        current_draw_dt = driver.find_element(By.XPATH, "//dt[contains(., 'Current') and contains(., 'Draw')]")
        parent_dl = current_draw_dt.find_element(By.XPATH, "./..")
        dd_elements = parent_dl.find_elements(By.TAG_NAME, "dd")
        current_draw_list = [dd.text.strip() for dd in dd_elements if dd.text.strip()]
        power_requirements = " | ".join(current_draw_list)
    except Exception as e:
        print(f"Power requirements not found for {url}: {e}") 
        power_requirements = ""

    # --------------------------------------- MODULE TAGS ---------------------------------------
    try:
        tags_div = driver.find_element(By.CSS_SELECTOR, "div.module-tags")
        tag_spans = tags_div.find_elements(By.CSS_SELECTOR, "span.label")
        module_tags = ", ".join([span.text.strip() for span in tag_spans if span.text.strip()])
    except Exception as e:
        print(f"Module tags not found for {url}: {e}")
        module_tags = ""

    # --------------------------------------- FULL DESCRIPTION ---------------------------------------
    try:
        module_details_div = driver.find_element(By.ID, "module-details")
        p_elements = module_details_div.find_elements(By.TAG_NAME, "p")
        full_desc_paragraphs = []

        for p in p_elements:
            p_classes = p.get_attribute("class") or ""
            
            if "lead" in p_classes and "wrap" in p_classes:
                continue
            if p.find_elements(By.TAG_NAME, "a"):
                continue
            text = p.text.strip()
            if text:
                full_desc_paragraphs.append(text)
        
        full_desc = "\n".join(full_desc_paragraphs)
    except Exception as e:
        print(f"Full description not found for {url}: {e}")
        full_desc = ""

    # --------------------------------------- PRICING INFORMATION (EURO) ---------------------------------------
    try:
        price_dd = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd")
        price_spans = price_dd.find_elements(By.XPATH, ".//span[contains(@class, 'currency-approx') or contains(@class, 'currency')]")
        price_texts = []

        for span in price_spans:
            text = span.text.strip()
            classes = span.get_attribute("class")

            if "currency-approx" in classes:
                text = "≈" + text
            price_texts.append(text)
        price_in_euro = " | ".join(price_texts)
    except Exception as e:
        print(f"Euro price not found for {url}: {e}")
        price_in_euro = ""

    # --------------------------------------- SWITCH TO USD AND SCRAPE PRICING ---------------------------------------
    try:
        usd_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@title, 'Display prices in $')]")))
        driver.execute_script("arguments[0].scrollIntoView();", usd_link)
        driver.execute_script("arguments[0].click();", usd_link)
        wait.until(lambda d: "$" in d.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd").text)

        price_dd_usd = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd")
        price_spans_usd = price_dd_usd.find_elements(By.XPATH, ".//span[contains(@class, 'currency-approx') or contains(@class, 'currency')]")
        price_texts_usd = []

        for span in price_spans_usd:
            text = span.text.strip()
            classes = span.get_attribute("class")

            if "currency-approx" in classes:
                text = "≈" + text
            price_texts_usd.append(text)
        price_in_dollar = " | ".join(price_texts_usd)
    except Exception as e:
        print(f"Dollar price not found for {url}: {e}")
        price_in_dollar = ""
    
    # --------------------------------------- REVERT TO EURO CURRENCY ---------------------------------------
    try:
        euro_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@title, 'Display prices in €')]")))
        driver.execute_script("arguments[0].scrollIntoView();", euro_link)
        driver.execute_script("arguments[0].click();", euro_link)
        wait.until(lambda d: "€" in d.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd").text)
    except Exception as e:
        print(f"Could not click to display Euro prices for {url}: {e}")

    # --------------------------------------- IMAGE EXTRACTION AND UPLOAD ---------------------------------------
    try:
        g_image_div = driver.find_element(By.CLASS_NAME, "g-image")
        a_tag = g_image_div.find_element(By.TAG_NAME, "a")
        relative_image_url = a_tag.get_attribute("href")

        if relative_image_url.startswith("/"):
            base_url = "https://modulargrid.net/"
            full_image_url = base_url + relative_image_url
        else:
            full_image_url= relative_image_url
        
        upload_result = cloudinary.uploader.upload(full_image_url)
        cloudinary_url = upload_result.get("secure_url", "")
    except Exception as e:
        print(f"Error uploading image for {url}: {e}")
        cloudinary_url = ""

    # --------------------------------------- STORE DATA ---------------------------------------
    data.append({
        "id": id,
        "module_name": module_name,
        "manufacturer": manufacturer,
        "primary_desc": primary_description,
        "available": availability,
        "approved_stamp": approved_stamp,
        "physical_dim": dimensions,
        "power_req": power_requirements,
        "module_tags": module_tags,
        "full_desc": full_desc,
        "price_in_euro": price_in_euro,
        "price_in_dollar": price_in_dollar,
        "module_url": url,
        "cloudinary_url": cloudinary_url
    })
    

## Cloudinary Configuration

In [290]:
cloudinary.config(
    cloud_name = "dglh7onu3",
    api_key = "115324728487844",
    api_secret = "68taYQlneJG-JU2izvZsOY3L8sk"
)

## Selenium Webdriver

In [291]:
# ------------------------------------------------------------------------------
# 1. Set up Selenium WebDriver
# ------------------------------------------------------------------------------

chrome_options = Options() 
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

BASE_URL = "https://modulargrid.net"
SEARCH_URL = (
    "https://modulargrid.net/e/modules/browser?"
    "SearchName=&SearchVendor=&SearchFunction=&SearchSecondaryfunction=&SearchHeight=&SearchTe=&"
    "SearchTemethod=max&SearchBuildtype=a&SearchLifecycle=available&SearchSet=&SearchMarketplace=&"
    "SearchIsmodeled=0&SearchShowothers=0&SearchShowpanel=0&order=newest&direction=asc"
)

In [292]:
# ---------------------------------------
# 2. Load the initial search page
# ---------------------------------------

driver.get(SEARCH_URL)
time.sleep(3)

In [294]:
# ------------------------------------------------------------------------------
# 3. Scroll to load more modules
# ------------------------------------------------------------------------------

try:
    alphabetic_button = driver.find_element(By.CSS_SELECTOR, "a[data-search-order='alphabetic']")
    alphabetic_button.click()
    
except Exception as e:
    print("Alphabetic sort button not found or click failed:", e)

SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    try:
        wait_for_scroll(last_height, timeout=20)

    except Exception as e:
        print("No more new content loaded or timeout reached:", e)
        break

    last_height = driver.execute_script("return document.body.scrollHeight")

KeyboardInterrupt: 

In [295]:
# ---------------------------------------
# 4. Loop through each module to grab the link
# ---------------------------------------

module_data = []
box_modules = driver.find_elements(By.CLASS_NAME, "box-module")

for module in box_modules:
    try:
        module_id = module.get_attribute("data-module-id")
        link_element = module.find_element(By.TAG_NAME, "a")
        href = link_element.get_attribute("href")
        
        if href.startswith("https"):
            print(f"Found module ID {module_id} with link {href}")
            module_data.append({"id": module_id, "link": href})
    except Exception as e:
        print("No link found in this module:", e)

print("Total modules found:", len(module_data))
        

Found module ID 24577 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-aluminium
Found module ID 24578 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black
Found module ID 26787 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black--
Found module ID 26788 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black-pink
Found module ID 29024 with link https://modulargrid.net/e/orpho-8-step
Found module ID 37431 with link https://modulargrid.net/e/synthrotek-adapt-1-4
Found module ID 12311 with link https://modulargrid.net/e/addac-system-addac812vu
Found module ID 51117 with link https://modulargrid.net/e/after-later-audio-ffs
Found module ID 26539 with link https://modulargrid.net/e/error-instruments-indian-resonator-v2
Found module ID 46869 with link https://modulargrid.net/e/apollo-view-modular-iou
Found module ID 10136 with link https:/

In [299]:
# ------------------------------------------------------------------------------
# 5. Iterate through each module link to scrape attributes
# ------------------------------------------------------------------------------

data = []
skipped_urls = []

driver.set_page_load_timeout(10)
wait = WebDriverWait(driver, 10)

total_links = len(module_data)

for idx, module_info in enumerate(module_data, start=1):
    url = module_info['link']
    id = module_info['id']

    print(f"Scraping module {idx} of {total_links}")

    try:
        driver.get(url)
    except TimeoutException:
        print(f"Skipping {url} due to timeout")
        skipped_urls.append(url)
        continue

    print(url)

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".module-view-header h1")))
    except Exception as e:
        print(f"Timeout waiting for module name on {url}: {e}")

    #--------------------------------------- MODULE NAME ---------------------------------------
    try:
        module_name_element = driver.find_element(By.CSS_SELECTOR, ".module-view-header h1")
        module_name = module_name_element.text.strip()
    except Exception as e:
        print(f"Module name not found for {url}: {e}")
        module_name = ""

    # --------------------------------------- MANUFACTURER ---------------------------------------
    try:
        manufacturer_element = driver.find_element(By.CSS_SELECTOR, ".vendor-name")
        manufacturer = manufacturer_element.text.strip()
    except Exception as e:
        print(f"Manufacturer not found for {url}: {e}")
        manufacturer = ""

    # --------------------------------------- PRIMARY DESCRIPTION ---------------------------------------
    try:
        primary_description_element = driver.find_element(By.CSS_SELECTOR, "p.lead.wrap")
        primary_description = primary_description_element.text.strip()
    except Exception as e:
        print(f"Primary description not found for {url}: {e}")
        primary_description = ""

    # --------------------------------------- AVAILABILITY ---------------------------------------
    try:
        availability_elements = driver.find_elements(
            By.XPATH, 
            "//div[contains(@class, 'subspec')]/p[contains(@class, 'text-success') and contains(., 'currently available')]"
        )
        availability = 1 if availability_elements else 0
    except Exception as e:
        print(f"Error determining availability for {url}: {e}")
        availability = 0

    # --------------------------------------- APPROVED STAMP ---------------------------------------
    try:
        driver.find_element(By.CSS_SELECTOR, ".box-approved")
        approved_stamp = 1
    except Exception:
        approved_stamp = 0

    # --------------------------------------- PHYSICAL DIMENSIONS ---------------------------------------
    try:
        dims_dt = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Dimensions']")
        parent_dl = dims_dt.find_element(By.XPATH, "./..")
        dd_elements = parent_dl.find_elements(By.TAG_NAME, "dd")
        dimensions_list = [dd.text.strip() for dd in dd_elements if dd.text.strip()]
        dimensions = " | ".join(dimensions_list)
    except Exception as e:
        print(f"Dimensions not found for {url}: {e}")
        dimensions = ""

    # --------------------------------------- POWER REQUIREMENTS ---------------------------------------
    try:
        current_draw_dt = driver.find_element(By.XPATH, "//dt[contains(., 'Current') and contains(., 'Draw')]")
        parent_dl = current_draw_dt.find_element(By.XPATH, "./..")
        dd_elements = parent_dl.find_elements(By.TAG_NAME, "dd")
        current_draw_list = [dd.text.strip() for dd in dd_elements if dd.text.strip()]
        power_requirements = " | ".join(current_draw_list)
    except Exception as e:
        print(f"Power requirement not found for {url}: {e}")
        power_requirements = ""

    # --------------------------------------- MODULE TAGS ---------------------------------------
    try:
        tags_div = driver.find_element(By.CSS_SELECTOR, "div.module-tags")
        tag_spans = tags_div.find_elements(By.CSS_SELECTOR, "span.label")
        module_tags = ", ".join([span.text.strip() for span in tag_spans if span.text.strip()])
    except Exception as e:
        print(f"Module tags not found for {url}: {e}")
        module_tags = ""

    # --------------------------------------- FULL DESCRIPTION ---------------------------------------
    try:
        module_details_div = driver.find_element(By.ID, "module-details")
        p_elements = module_details_div.find_elements(By.TAG_NAME, "p")
        full_desc_paragraphs = []
        
        for p in p_elements:
            p_classes = p.get_attribute("class") or ""
            if "lead" in p_classes and "wrap" in p_classes:
                continue
            
            if p.find_elements(By.TAG_NAME, "a"):
                continue
            
            text = p.text.strip()
            if text:
                full_desc_paragraphs.append(text)
        
        full_desc = "\n".join(full_desc_paragraphs)
    except Exception as e:
        print(f"Full description not found for {url}: {e}")
        full_desc = ""

    # --------------------------------------- PRICING INFORMATION (EURO) ---------------------------------------
    try:
        price_dd = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd")
        price_spans = price_dd.find_elements(By.XPATH, ".//span[contains(@class, 'currency-approx') or contains(@class, 'currency')]")
        price_texts = []
        for span in price_spans:
            text = span.text.strip()
            classes = span.get_attribute("class")
            if "currency-approx" in classes:
                text = "≈" + text
            price_texts.append(text)
        price_in_euro = " | ".join(price_texts)
    except Exception as e:
        print(f"Euro price not found for {url}: {e}")
        price_in_euro = ""

    # --------------------------------------- SWITCH TO USD AND SCRAPE PRICING ---------------------------------------
    try:
        # Click the USD link to switch currency
        usd_link = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@title, 'Display prices in $')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", usd_link)
        driver.execute_script("arguments[0].click();", usd_link)

        # Wait for the price to update to USD (optional)
        wait.until(lambda d: "$" in d.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd").text)

        # Scrape USD pricing
        price_dd_usd = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd")
        price_spans_usd = price_dd_usd.find_elements(By.XPATH, ".//span[contains(@class, 'currency-approx') or contains(@class, 'currency')]")
        price_texts_usd = []
        for span in price_spans_usd:
            text = span.text.strip()
            classes = span.get_attribute("class")
            if "currency-approx" in classes:
                text = "≈" + text
            price_texts_usd.append(text)
        price_in_dollar = " | ".join(price_texts_usd)
    except Exception as e:
        print(f"Dollar price not found for {url}: {e}")
        price_in_dollar = ""

    # --------------------------------------- REVERT TO EURO CURRENCY ---------------------------------------
    try:
        eur_link = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@title, 'Display prices in €')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", eur_link)
        driver.execute_script("arguments[0].click();", eur_link)

        wait.until(lambda d: "€" in d.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd").text)
    except Exception as e:
        print(f"Could not click to display Euro prices for {url}: {e}")

    # --------------------------------------- IMAGE EXTRACTION AND UPLOAD ---------------------------------------
    try:
        g_image_div = driver.find_element(By.CLASS_NAME, "g-image")
        a_tag = g_image_div.find_element(By.TAG_NAME, "a")
        relative_image_url = a_tag.get_attribute("href")

        if relative_image_url.startswith("/"):
            base_url = "https://modulargrid.net/"
            full_image_url = base_url + relative_image_url
        else:
            full_image_url = relative_image_url
        
        upload_result = cloudinary.uploader.upload(full_image_url)
        cloudinary_url = upload_result.get("secure_url", "")
    except Exception as e:
        print(f"Error uploading image for {url}: {e}")
        cloudinary_url = ""


    # --------------------------------------- STORE DATA ---------------------------------------
    data.append({
        "id": id,
        "module_name": module_name,
        "manufacturer": manufacturer,
        "primary_desc": primary_description,
        "available": availability,
        "approved_stamp": approved_stamp,
        "physical_dim": dimensions,
        "power_req": power_requirements,
        "module_tags": module_tags,
        "full_desc": full_desc,
        "price_in_euro": price_in_euro,
        "price_in_dollar": price_in_dollar,
        "module_url": url,
        "cloudinary_url": cloudinary_url
    })


Scraping module 1 of 600
https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-aluminium
Scraping module 2 of 600
https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black
Scraping module 3 of 600
https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black--
Scraping module 4 of 600
https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black-pink
Scraping module 5 of 600
https://modulargrid.net/e/orpho-8-step
Scraping module 6 of 600
https://modulargrid.net/e/synthrotek-adapt-1-4
Scraping module 7 of 600
https://modulargrid.net/e/addac-system-addac812vu
Scraping module 8 of 600
https://modulargrid.net/e/after-later-audio-ffs
Scraping module 9 of 600
https://modulargrid.net/e/error-instruments-indian-resonator-v2
Scraping module 10 of 600
https://modulargrid.net/e/apollo-view-modular-iou
Scraping module 11 of 600
https://modulargrid.net/e/seismic-industries-ips-interruptible-power-supply
Scrapi

In [ ]:
# ------------------------------------------------------------------------------
# 6. Iterate through skipped links (due to timeout) to scrape attributes
# ------------------------------------------------------------------------------

if skipped_urls:
    print("\n Processing skipped URLs separately without timeout constraints. \n")

    driver.set_page_load_timeout(0)

    total_skipped = len(skipped_urls)
    
    for idx, module_info in enumerate(skipped_urls, start=1):
        url = module_info['link']
        module_id = module_info['id']

        print(f"Processing skipped URL {idx} of {total_skipped}: {url}")
        driver.get(url)

        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".module-view-header h1")))
        except Exception as e:
            print(f"Error waiting for header on {url}: {e}")

In [301]:
skipped_urls

['https://modulargrid.net/e/ph-modular-2hp-3xatt',
 'https://modulargrid.net/e/ph-modular-2hp-drive',
 'https://modulargrid.net/e/ph-modular-2hp-multiple',
 'https://modulargrid.net/e/ph-modular-2hp-patchbay',
 'https://modulargrid.net/e/ph-modular-2x-a-b',
 'https://modulargrid.net/e/ph-modular-2x-a-b-alternative-version',
 'https://modulargrid.net/e/ph-modular-3x-att']

In [300]:
df = pd.DataFrame(data)
df

,id,module_name,manufacturer,primary_desc,available,approved_stamp,physical_dim,power_req,module_tags,full_desc,price_in_euro,price_in_dollar,module_url,cloudinary_url
0,24577,"""РИТМИКС"" aluminium",Paratek,8 channels mixer unit,1,1,12 HP | 38 mm deep,40 mA +12V | 5 mA -12V | ? mA 5V,"Attenuator, Dual/Stereo, Mixer, Panning",• 8 independent mono channels for audio signal...,≈€260,≈$289,https://modulargrid.net/e/paratek-%D0%A0%D0%98...,https://res.cloudinary.com/dglh7onu3/image/upl...
1,24578,"""РИТМИКС"" black",Paratek,8 channels mixer unit,1,1,12 HP | 38 mm deep,40 mA +12V | 5 mA -12V | ? mA 5V,"Attenuator, Dual/Stereo, Mixer, Multiple, Panning",• 8 independent mono channels for audio signal...,≈€260,≈$289,https://modulargrid.net/e/paratek-%D0%A0%D0%98...,https://res.cloudinary.com/dglh7onu3/image/upl...
2,26787,"""РИТМИКС"" black",Paratek,8 channels mixer unit,1,1,12 HP | 38 mm deep,40 mA +12V | 5 mA -12V | ? mA 5V,"Attenuator, Dual/Stereo, Mixer, Multiple, Panning",• 8 independent mono channels for audio signal...,≈€260,≈$289,https://modulargrid.net/e/paratek-%D0%A0%D0%98...,https://res.cloudinary.com/dglh7onu3/image/upl...
3,26788,"""РИТМИКС"" black (pink)",Paratek,8 channels mixer unit,1,1,12 HP | 38 mm deep,40 mA +12V | 5 mA -12V | ? mA 5V,"Attenuator, Dual/Stereo, Mixer, Multiple, Panning",• 8 independent mono channels for audio signal...,≈€260,≈$289,https://modulargrid.net/e/paratek-%D0%A0%D0%98...,https://res.cloudinary.com/dglh7onu3/image/upl...
4,29024,8-Step,Orpho,TRIGGER SEQUENCER,1,0,6 HP | 35 mm deep,50 mA +12V | ? mA -12V | ? mA 5V,Sequencer,The Orpho 8-Step is an eight-step sequencer wi...,€85,≈$92,https://modulargrid.net/e/orpho-8-step,https://res.cloudinary.com/dglh7onu3/image/upl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,7765,A-110-2V,Doepfer,Basic VCO Vintage Edition (black front panel),1,0,8 HP | 50 mm deep,150 mA +12V | 30 mA -12V | 0 mA 5V,Oscillator,Module A-110-2 is a low-cost voltage-controlle...,€140,≈$155,https://modulargrid.net/e/doepfer-a-110-2v,https://res.cloudinary.com/dglh7onu3/image/upl...
589,4953,A-110-4,Doepfer,Quadrature Thru Zero Voltage Controlled Oscill...,1,0,8 HP | 60 mm deep,90 mA +12V | 30 mA -12V | 0 mA 5V,Oscillator,"A-110-4 is a special audio VCO. The term ""quad...",€140,$160,https://modulargrid.net/e/doepfer-a-110-4,https://res.cloudinary.com/dglh7onu3/image/upl...
590,6855,A-110-4 SE,Doepfer,Quadrature Thru Zero Voltage Controlled Oscill...,1,0,8 HP | 60 mm deep,90 mA +12V | 30 mA -12V | 0 mA 5V,Oscillator,A-110-4 is a so-called Thru Zero Quadrature VC...,€140,$175,https://modulargrid.net/e/doepfer-a-110-4-se,https://res.cloudinary.com/dglh7onu3/image/upl...
591,7771,A-110-6,Doepfer,Trapezoid Thru Zero Quadrature VCO,1,0,12 HP | 61 mm deep,80 mA +12V | 70 mA -12V | ? mA 5V,"Oscillator, LFO",A-110-4 is a Trapezoid Thru Zero Quadrature VC...,€250,$250,https://modulargrid.net/e/doepfer-a-110-6,https://res.cloudinary.com/dglh7onu3/image/upl...


In [ ]:
print(df)

In [ ]:
df['available'].value_counts()

In [287]:
df.to_csv("full_sample.csv", index=False)